In [1]:
import ee
from pathlib import Path
import pandas as pd
import ast

ee.Authenticate() #Uncomment this whenever needed, once done usually not needed for 1-2 days
ee.Initialize(project='ee-raman')

In [2]:
def get_feature_collection(asset_id):
    """Check if an asset exists, and load it as a FeatureCollection if it does.
    Otherwise, return an empty FeatureCollection.
    
    Args:
        asset_id (str): The Earth Engine asset ID.
        
    Returns:
        ee.FeatureCollection: The loaded FeatureCollection or an empty one.
    """
    try:
        # Get asset information to check existence
        ee.data.getAsset(asset_id)
        print(f"Asset '{asset_id}' exists. Loading FeatureCollection.")
        return ee.FeatureCollection(asset_id)
    except Exception as e:
        print(f"Asset '{asset_id}' does not exist. Returning empty FeatureCollection.")
        return ee.FeatureCollection([])

In [3]:
import math
from itertools import product


# Function to convert latitude to pixel Y at a given zoom level
def lat_to_pixel_y(lat, zoom):
    sin_lat = math.sin(math.radians(lat))
    pixel_y = ((0.5 - math.log((1 + sin_lat) / (1 - sin_lat)) / (4 * math.pi)) * (2 ** (zoom + 8)))
    return pixel_y

# Function to convert longitude to pixel X at a given zoom level
def lon_to_pixel_x(lon, zoom):
    pixel_x = ((lon + 180) / 360) * (2 ** (zoom + 8))
    return pixel_x

# Function to convert pixel X to longitude
def pixel_x_to_lon(pixel_x, zoom):
    lon = (pixel_x / (2 ** (zoom + 8))) * 360 - 180
    return lon

# Function to convert pixel Y to latitude
def pixel_y_to_lat(pixel_y, zoom):
    n = math.pi - 2 * math.pi * pixel_y / (2 ** (zoom + 8))
    lat = math.degrees(math.atan(math.sinh(n)))
    return lat

def lat_lon_from_pixel(lat, lon, zoom, scale):
    """
    Given a starting latitude and longitude, calculate the latitude and longitude
    of the opposite corner of a 256x256 image at a given zoom level.
    """
    pixel_x = lon_to_pixel_x(lon, zoom)
    pixel_y = lat_to_pixel_y(lat, zoom)
    
    new_lon = pixel_x_to_lon(pixel_x + 256*scale, zoom)
    new_lat = pixel_y_to_lat(pixel_y + 256*scale, zoom)

    return new_lat, new_lon

"""

Helper function for dividing an roi into blocks

"""
def get_n_boxes(lat, lon, n, zoom, scale):
    diagonal_lat_lon = [(lat, lon),]
    for i in range(n):
        new_lat_lon = lat_lon_from_pixel(lat, lon, zoom, scale)
        diagonal_lat_lon.append(new_lat_lon)
        lat, lon = new_lat_lon
    lats = [i[0] for i in diagonal_lat_lon]
    longs = [i[1] for i in diagonal_lat_lon]
    return list(product(lats, longs))

def get_points(roi, directory):
    Path(directory).mkdir(parents=True, exist_ok=True)
    points_file = Path(directory + "/status.csv")
    if points_file.is_file():
        df = pd.read_csv(directory + "/status.csv", index_col=False)
        df["points"] = df['points'].apply(ast.literal_eval)
        return df
    zoom = 17
    scale = 16
    bounds = roi.bounds().coordinates().get(0).getInfo()
    lons = sorted([i[0] for i in bounds])
    lats = sorted([i[1] for i in bounds])
    starting_point = lats[-1], lons[0]
    min_, max_ = (
        [lon_to_pixel_x(lons[0], zoom), lat_to_pixel_y(lats[0], zoom) ],
        [lon_to_pixel_x(lons[-1], zoom), lat_to_pixel_y(lats[-1], zoom)]
        )
    iterations = math.ceil(max(abs(min_[0] -  max_[0]), abs(min_[1] - max_[1]))/256/16)
    points = get_n_boxes(starting_point[0], starting_point[1], iterations, zoom, scale)
    intersect_list = []
    print(len(points))
    index = 0
    for point in points:
        top_left = point
        bottom_right = lat_lon_from_pixel(top_left[0], top_left[1], zoom, scale)
        rectangle = ee.Geometry.Rectangle([(top_left[1], top_left[0]), (bottom_right[1], bottom_right[0])])
        print(top_left, bottom_right)
        intersects = roi.geometry().intersects(rectangle, ee.ErrorMargin(1)).getInfo()
        if intersects:
            intersect_list.append((index, (top_left,bottom_right)))
            index+=1
        print(intersects)
    df = pd.DataFrame(intersect_list, columns=["index", "points"])
    df["overall_status"] = False
    df["download_status"] = False
    df["model_status"] = False
    df["segmentation_status"] = False
    df["postprocessing_status"] = False
    df["plantation_status"] = False
    df.to_csv(directory + "/status.csv", index=False)
    return df

state = "karnataka"
district = "raichur"
block = "devadurga"

state = "jharkhand" # have same error
district = "dumka"
block = "masalia"

state = "odisha" # Local computer not run
district = "anugul"
block = "anugul"

roi_boundary_original = ee.FeatureCollection("projects/ee-corestackdev/assets/apps/mws/"+state+"/"+district+"/"+block+"/filtered_mws_"+district+"_"+block+"_uid")


blocks_df = get_points(roi_boundary_original, block)
points = list(blocks_df["points"])

roi_boundary = ee.FeatureCollection([ee.Feature(ee.Geometry.Rectangle([top_left[1], bottom_right[0], bottom_right[1], top_left[0]])) for top_left, bottom_right in points])


In [4]:
mapping = {
    "farm": 1,
    "plantation": 2,
    "scrubland": 3,
    "rest": 0
}
reversed_mapping = {v: k for k, v in mapping.items()}
reversed_ee_mapping = ee.Dictionary(reversed_mapping)

easy_farm = [
    ee.Filter.gte("rect", 0.67),
    ee.Filter.gt("size", 500),
    ee.Filter.lt("size", 2000),
    ee.Filter.lt("ent", 1)
    ]

easy_scrubland = [
    ee.Filter.gte("size", 60000)
    ]

asset_string = state + "/" + district + "/" + block + "/" + district + "_" + block

all = get_feature_collection("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_boundaries")
farm = all.filter(ee.Filter.And(*easy_farm))

#Filter out farms which doesnot have 3 nearby farms (Removing solo farms inside scrublands)
farm_buffer = farm.map(lambda x: x.buffer(10))
farm_image = ee.Image(0)
farm_mask = farm_image.clip(farm_buffer).mask()
farm_vectors = farm_mask.toInt().reduceToVectors(
    geometry=roi_boundary,
    scale=10,  # Change based on your resolution
    geometryType='polygon',
    labelProperty='zone',
    reducer=ee.Reducer.countEvery(),
    maxPixels=1e8
) 
farm_vectors = farm_vectors.filter(ee.Filter.eq('zone', 1)).map(lambda x: x.set("count", farm.filterBounds(x.geometry()).size())).filter(ee.Filter.gt('count', 3))

Asset 'projects/ee-corestackdev/assets/apps/mws/odisha/anugul/anugul/anugul_anugul_boundaries' exists. Loading FeatureCollection.


In [5]:
task = ee.batch.Export.table.toAsset(
    collection=farm_vectors,
    description='Farm_Clustering',
    assetId="projects/ee-raman/assets/" + block + "_farm_clusters"
)

# Start the task
task.start()

In [5]:
## Scrubland filtering. Remove large boundaries due to wrong field segmentation from scrubland samples
# Step 1: Mask for class 8, 9, 10, 11
scrubland = all.filter(ee.Filter.And(easy_scrubland))
lulc_v3 = ee.Image("projects/ee-corestackdev/assets/apps/mws/" + asset_string + "_2023-07-01_2024-06-30_LULCmap_10m")
classes_of_interest = [8, 9, 10, 11]
masked_lulc = lulc_v3.remap(classes_of_interest, [1]*len(classes_of_interest))  # 1 where class is of interest, 0 elsewhere

# Step 2: Function to compute % area of interest inside each feature
def filter_by_lulc(feature):
    geom = feature.geometry()
    scale = 30  # Set resolution appropriate to your LULC data

    # Area of interest within polygon (masked_lulc == 1)
    interest_area_img = ee.Image.pixelArea().updateMask(masked_lulc)
    interest_area = interest_area_img.reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=scale,
        maxPixels=1e8
    ).get('area')

    # Total area of the polygon
    total_area = ee.Image.pixelArea().reduceRegion(
        reducer=ee.Reducer.sum(),
        geometry=geom,
        scale=scale,
        maxPixels=1e8
    ).get('area')

    # Compute % and add it as a property
    percent_interest = ee.Number(interest_area).divide(ee.Number(total_area)).multiply(100)
    return feature.set('percent_interest', percent_interest)

# Step 3: Apply to FeatureCollection
scrubland = scrubland.map(filter_by_lulc)

# Step 4: Filter features with > 50% area of interest
def strictly_inside_roi(feature):
    return feature.set('inside', roi_boundary_original.union().geometry().contains(feature.geometry()))

scrubland = scrubland.filter(ee.Filter.lt('percent_interest', 50)).map(strictly_inside_roi)
scrubland = scrubland.filter(ee.Filter.eq('inside', True))


In [6]:
task = ee.batch.Export.table.toAsset(
    collection=scrubland,
    description='Scrubland_Filtering',
    assetId="projects/ee-raman/assets/" + block + "_filtered_scrublands"
)

# Start the task
task.start()